# 几何畸变校正探索

In [1]:
import os,tarfile,sys
import ee
import geemap
import numpy as np
import pandas as pd
import eerepr
import folium
import copy
import math
from tqdm import trange,tqdm
from functools import partial
import geopandas as gpd
geemap.set_proxy(port=10809)
ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))


Successfully saved authorization token.
geemap version = 0.17.2
geemap path = ['D:\\Code_base\\anaconda\\envs\\s1s2cloud_rm\\lib\\site-packages\\geemap']


# 预加载SHP数据

In [2]:
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))
    #projects/ee-mrwurenzhe/assets/Glacial_lake/Checkout_polygen

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())

Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

# Slop correction

In [3]:
#-----------------------------------------------地形校正-----------------------------------------
def slope_correction(collection, elevation, model, buffer=0):
    '''This function applies the slope correction on a collection of Sentinel-1 data

       :param collection: ee.Collection or ee.Image of Sentinel-1
       :param elevation: ee.Image of DEM
       :param model: model to be applied (volume/surface)
       :param buffer: buffer in meters for layover/shadow amsk

       :returns: ee.Image
    '''
    def _volumetric_model_SCF(theta_iRad, alpha_rRad):
        '''Code for calculation of volumetric model SCF
            体积模型
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range

        :returns: ee.Image
        '''

        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

        # model
        nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
        denominator = (ninetyRad.subtract(theta_iRad)).tan()
        return nominator.divide(denominator)

    def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
        '''Code for calculation of direct model SCF
            表面模型
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        :param alpha_azRad: ee.Image of slope steepness in azimuth

        :returns: ee.Image
        '''

        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

        # model
        nominator = (ninetyRad.subtract(theta_iRad)).cos()
        denominator = (alpha_azRad.cos().multiply(
            (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()))

        return nominator.divide(denominator)

    def _erode(image, distance):
        '''Buffer function for raster
            腐蚀算法，输入的图像需要额外的缓冲
        :param image: ee.Image that shoudl be buffered
        :param distance: distance of buffer in meters

        :returns: ee.Image
        '''

        d = (image.Not().unmask(1).fastDistanceTransform(30).sqrt().multiply(
            ee.Image.pixelArea().sqrt()))

        return image.updateMask(d.gt(distance))

    def _masking(alpha_rRad, theta_iRad, buffer):
        '''Masking of layover and shadow
            获取几何畸变区域
        :param alpha_rRad: ee.Image of slope steepness in range
        :param theta_iRad: ee.Image of incidence angle in radians
        :param buffer: buffer in meters

        :returns: ee.Image
        '''
        # layover, where slope > radar viewing angle
        layover = alpha_rRad.lt(theta_iRad).rename('layover')

        # shadow
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
        shadow = alpha_rRad.gt(
            ee.Image.constant(-1).multiply(
                ninetyRad.subtract(theta_iRad))).rename('shadow')

        # add buffer to layover and shadow
        if buffer > 0:
            layover = _erode(layover, buffer)
            shadow = _erode(shadow, buffer)

        # combine layover and shadow
        no_data_mask = layover.And(shadow).rename('no_data_mask')

        return layover.addBands(shadow).addBands(no_data_mask)

    def _correct(image):
        '''This function applies the slope correction and adds layover and shadow masks

        '''

        # get the image geometry and projection
        geom = image.geometry()
        proj = image.select(1).projection()

        # calculate the look direction
        heading = (ee.Terrain.aspect(image.select('angle')).reduceRegion(
            ee.Reducer.mean(), geom, 1000).get('aspect'))

        # Sigma0 to Power of input image，Sigma0是指雷达回波信号的强度
        sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

        # the numbering follows the article chapters
        # 2.1.1 Radar geometry
        theta_iRad = image.select('angle').multiply(np.pi / 180)
        phi_iRad = ee.Image.constant(heading).multiply(np.pi / 180)

        # 2.1.2 Terrain geometry
        alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(
            np.pi / 180).setDefaultProjection(proj).clip(geom)
        phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(
            np.pi / 180).setDefaultProjection(proj).clip(geom)

        # we get the height, for export
        height = elevation.setDefaultProjection(proj).clip(geom)

        # 2.1.3 Model geometry
        #reduce to 3 angle
        phi_rRad = phi_iRad.subtract(phi_sRad)

        # slope steepness in range (eq. 2)
        alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

        # slope steepness in azimuth (eq 3)
        alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

        # local incidence angle (eq. 4)
        theta_liaRad = (alpha_azRad.cos().multiply(
            (theta_iRad.subtract(alpha_rRad)).cos())).acos()
        theta_liaDeg = theta_liaRad.multiply(180 / np.pi)

        # 2.2
        # Gamma_nought
        gamma0 = sigma0Pow.divide(theta_iRad.cos())
        gamma0dB = ee.Image.constant(10).multiply(gamma0.log10()).select(
            ['VV', 'VH'], ['VV_gamma0', 'VH_gamma0'])
        ratio_gamma = (gamma0dB.select('VV_gamma0').subtract(
            gamma0dB.select('VH_gamma0')).rename('ratio_gamma0'))

        if model == 'volume':
            scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

        if model == 'surface':
            scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

        # apply model for Gamm0_f
        gamma0_flat = gamma0.divide(scf)
        gamma0_flatDB = (ee.Image.constant(10).multiply(
            gamma0_flat.log10()).select(['VV', 'VH'],
                                        ['VV_gamma0flat', 'VH_gamma0flat']))

        masks = _masking(alpha_rRad, theta_iRad, buffer)

        # calculate the ratio for RGB vis
        ratio_flat = (gamma0_flatDB.select('VV_gamma0flat').subtract(
            gamma0_flatDB.select('VH_gamma0flat')).rename('ratio_gamma0flat'))

        return (image.rename(['VV_sigma0', 'VH_sigma0','incAngle'])
                .addBands(gamma0dB)
                .addBands(ratio_gamma)
                .addBands(gamma0_flatDB)
                .addBands(ratio_flat)
                .addBands(alpha_rRad.rename('alpha_rRad'))
                .addBands(alpha_azRad.rename('alpha_azRad'))
                .addBands(phi_sRad.rename('aspect'))
                .addBands(alpha_sRad.rename('slope'))
                .addBands(theta_iRad.rename('theta_iRad'))
                .addBands(theta_liaRad.rename('theta_liaRad'))
                .addBands(masks).addBands(height.rename('elevation')))

    # run and return correction
    if type(collection) == ee.imagecollection.ImageCollection:
        return collection.map(_correct)
    elif type(collection) == ee.image.Image:
        return _correct(collection)
    else:
        print('Check input type, only image collection or image can be input')

# 定义数据

In [4]:
START_DATE  = ee.Date('2019-07-01')
END_DATE   = ee.Date('2019-08-30')
DEM = ee.Image('USGS/SRTMGL1_003')
models = ['volume', 'surface', None]
Buffer = 0
Model = models[0]
I = 100

AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(I))

s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
          .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filterBounds(AOI)
            .filterDate(START_DATE, END_DATE))

# 几何畸变定位与校正

In [5]:
s1_col = slope_correction(s1_col, DEM, Model, buffer=Buffer)
# 分离升降轨
s1_descending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).first()
s1_ascending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first()

## 展示,生成左右对照Basemap与畸变检测对照

In [6]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=12)
left_layer = geemap.ee_tile_layer(s1_ascending.select('VV_sigma0'), {'min':-40,'max':5}, 'Origin')
right_layer =  geemap.ee_tile_layer(s1_ascending.select('no_data_mask'), {'min':0,'max':1}, 'Correct')
Map.split_map(left_layer,right_layer)
Map

Map(center=[28.752624732215644, 94.00131716748263], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [7]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=12)
left_layer = geemap.ee_tile_layer(s1_descending, {'min':-40,'max':5}, 'NLCD 2001')
Map.split_map(left_layer,left_layer)
Map

Map(center=[28.752624732215644, 94.00131716748263], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [8]:
# 被动阴影和叠掩

In [9]:
## 沿着雷达入射角，构建近似于等高线，在登高线方向做局部最高点，且在坡两端做法线夹角

In [10]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=12)

# 设置离散的颜色调色板列表（以角度为单位）
palette = ['blue', 'green', 'yellow', 'red']
MinMax_Angle = s1_ascending.select('incAngle').reduceRegion(
                  reducer=ee.Reducer.minMax(),
                  geometry=s1_ascending.geometry(),
                  scale=10,
                  bestEffort=True
                                )
min =  MinMax_Angle.get('incAngle_min')
max = MinMax_Angle.get('incAngle_max')
interval = 1

discrete_vis = {'min':min, 'max':max ,'palette': palette}
contours = geemap.create_contours(s1_ascending.select('incAngle'),min ,max, interval, region=None)

Map.addLayer(contours, {'min': min, "max":max, 'color':'black'}, 'contours')
Map.addLayer(s1_ascending.select('incAngle'), discrete_vis, 'incAngle')

Map # 显示地图窗口

Map(center=[28.752624732215644, 94.00131716748263], controls=(WidgetControl(options=['position', 'transparent_…